# doc2vecを使い、livedoor newsコーパスをベクトル化する

1. mecabで分かち書きする
1. doc2vecに学習させ、ベクトルを取得する
1. 似ている文書を探してみる
1. ベクトル化したデータをtensorflow形式に変換して保存する


doc2vecについてはここが詳しい

https://deepage.net/machine_learning/2017/01/08/doc2vec.html

In [1]:
from ipywidgets.widgets import FloatProgress
from IPython.display import display

import random
import sys
from os import listdir, path, makedirs
import numpy as np
import MeCab
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import LabeledSentence

mecab = MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')

### ディレクトリ内のファイル一覧を取得する

Dockerfileでlivedoor newsコーパスをダウンロードしたパスを参照しファイルの一覧を配列で取得する

In [2]:
dirs = [{'key':i, 'label':path.join('../data/text', x)} for i, x in enumerate(listdir('../data/text')) if not x.endswith('.txt')]
dir_docs = [[path.join(x['label'], y), i, x['label'], x['key']] for x in dirs for i, y in enumerate(listdir(x['label']))]

print(dir_docs[:3])
print(len(dir_docs))

[['../data/text/peachy/peachy-6285686.txt', 0, '../data/text/peachy', 0], ['../data/text/peachy/peachy-4822718.txt', 1, '../data/text/peachy', 0], ['../data/text/peachy/peachy-4754128.txt', 2, '../data/text/peachy', 0]]
7376


### mapfileを保存しておく (使い方よくわかってない)

Doc2Vecの引数で渡すファイルであるが、idともとの文書を紐付けるために使う事もできる

In [3]:
with open('../input/mapfile.txt', 'w') as f:
    f.write("original file,id,category name,category id\n")
    for i, doc in enumerate(dir_docs):
        doc[1] = i
        f.write("{}:{},{},{}\n".format(doc[0], doc[1], doc[2], doc[3]))

### 全ファイル分かち書きしてLabeldSentenceを作る

https://radimrehurek.com/gensim/models/doc2vec.html

In [4]:
training_docs = []
fp = FloatProgress(min=0, max=len(dir_docs))
display(fp)
for idx, (dir_doc) in enumerate(dir_docs):
    text = ''
    for line in open(dir_doc[0], 'r'):
        if (line is ''):
            continue

        text += mecab.parse(line)

    # doc2vecのタグは一意なもの(ファイル名)にしておく
    training_docs.append(LabeledSentence(words=text, tags=[dir_doc[0]]))
    
    fp.value = idx

### 学習させる
dm=0 : DBoWで学習
dm=1 : dmpv(Distributed Memory)で学習

少し時間がかかる

HACK: dmpvのほうが制度が高いらしいのでいろいろ試してみたい

In [5]:
model = Doc2Vec(documents=training_docs, min_count=1, dm=0, docvecs_mapfile="../input/mapfile.txt")

### 文書を入力して、近い文書を取得する

most_similarメソッドに文書タグを渡すと、一致したタグの文書と最も近いタグを探してくれる

結果はdictになり類似度が数値で入ってくるので一番高いやつのタグを見ておこう

In [6]:
print(open('../data/text/dokujo-tsushin/dokujo-tsushin-4842348.txt').read())

print(model.docvecs.most_similar('../data/text/dokujo-tsushin/dokujo-tsushin-4842348.txt'))


http://news.livedoor.com/article/detail/4842348/
2010-06-25T14:30:00+0900
結婚相手に妥協できるもの、できないもの　（男性編）
　婚カツが日常化した昨今、男女ともども結婚相手を求めてさまよう毎日。しかし何度もパーティーや合コンに通っても、なかなか理想の異性に出会えないのも現実。やはり一生を添い遂げる相手には“ここは譲れない”モノがそれぞれにあるのだろう。

　では反対に、結婚相手に“まあ妥協できるかな”と思えるのは一体どんな事なのだろうか？

　ライブドアの結婚紹介サービス ユーブライド詳細はコチラ で行った「あなたが結婚するときに妥協できるところは？」のアンケートでは、男性の“妥協できる、できない”点に意外な結果を見ることができた。

　アンケートは、男性が結婚相手に“妥協”できるものを身長、顔、職業、服のセンス、親との同居、価値観、年収、転勤、金銭感覚、車の有無、家族構成、離婚歴、学歴、喫煙、ギャンブルの16項目の中から複数回答で選択する方式で1000人以上の会員によって回答されたものだ。

　男性のアンケートの結果を見ると、女性に“妥協”できる項目には3位に車の有無、2位に身長、そして意外にも1位が離婚歴だった。離婚した夫婦が周りに当たり前にいる現在では、バツイチくらいでは“妥協”にもならないというのが本音なのだろうか。

　また“妥協できる条件に選ばれなかった”項目では、3位が親との同居、2位に価値観、そして金銭感覚が1位の結果に。この金銭感覚については、20代前半から40代前半の男性すべてが“妥協できない”の項目1位に挙げているところがおもしろい。

　さらに世代別で見てみると、20代男性の“妥協できる”上位には顔や身長、年収などが入り、30代〜60代では、首位の離婚歴のほか、30代では車の有無や年収、40代以上では家族構成も上位にランクインしている。

　世代別の“妥協できる条件に選ばれなかった”のは、20代では服のセンス、30代、40代の働き盛りの男性では転勤も上位に入る結果に。

　この結果で分かるのは、男性全体が金銭感覚も含めて価値観が一緒の女性を求めているということ。そして20代ではセンス、30代以降では容姿を重要視していることも分かる。

　このアンケートを踏まえて、婚カツ

### 結果確認

most_similarで一番近いと言われたタグがどんな文書なのか確認してみる

In [7]:
print(open('../data/text/dokujo-tsushin/dokujo-tsushin-4887920.txt').read())

http://news.livedoor.com/article/detail/4887920/
2010-07-16T14:00:00+0900
結婚相手に妥協できるものできないもの（女性編）
ライブドアの結婚紹介サービスユーブライドで行った「あなたが結婚するときに妥協できるところは？」のアンケート実施にて、男性が結婚相手に妥協できるもの、できないものに意外な解答があったことについて前回紹介したが、女性のアンケート回答でも納得できるもの、えっそうなのと驚く内容となっていたので紹介したい。

アンケートは、結婚相手に“妥協”できるものを「身長、顔、職業、服のセンス、親との同居、価値観、年収、転勤、金銭感覚、車の有無、家族構成、離婚歴、学歴、喫煙、ギャンブル」の16項目の中から複数回答で選択する方式で1000人以上の会員によって回答されたもので、男性に実施したものと同様。

アンケート結果では、女性が結婚相手の男性に“妥協できる”項目としては、最も妥協できる1位の離婚歴と2位の身長は、男性の結果と同じだが、3位には喫煙がランクインしている。

分煙が当たり前なこの世の中、女性は喫煙を嫌っている傾向があるように思えるが、意外なことに20代前半から40代後半の女性までに「喫煙」は妥協できる上位に入っており、反対に50代女性になると、妥協できる項目ではなくなるようだ。

また、男性と女性の回答で違いをみせたのが「転勤」。男性には妥協できるものには入らない項目であったが、女性とってはそれほどでもないのか、20代前半と40代以降の女性では妥協できる項目の上位であった。

さらに女性のシビアな面を見せたのが「お金」に関する項目だ。「金銭感覚」と「収入」の項目は女性にとって一番妥協したくないようで、どの年代でも妥協できる条件には選ばれなかった。

このアンケートを総合してみると、女性は結婚相手には「美意識」や「外見」にはあまりこだわっていないように思える。が、しかし、生活の安定に重視していることが分かる。

女性にとって「高学歴」「高身長」「高収入」の3高のといった、夢のような男性は遠い過去なのだ。「就活」「婚カツ」できびしい現実を経験した女性にとって「結婚」＝「安定」が何より欲しい幸福なのかも知れない。



### 単語ベクトルを参照する

単語を一覧で取得し、それぞれの単語のベクトルを参照してみる

[参考にしたページ](http://wiki.ruka-f.net/index.php?word2vec)

In [8]:
matrix = np.array([ model.wv[word] for word in model.wv.vocab.keys()])
print(matrix)
print(matrix.shape)

[[ 0.00027593  0.00074948 -0.00272737 ..., -0.00070485 -0.00016166
   0.00071484]
 [-0.00395702 -0.00465271  0.0045014  ...,  0.00261668 -0.00074409
  -0.00110428]
 [ 0.0004709  -0.00372458  0.0004484  ...,  0.00039862  0.00163544
   0.00226335]
 ..., 
 [-0.00412573  0.00186599  0.00276733 ..., -0.00346943 -0.00265568
   0.0014411 ]
 [ 0.0039717   0.0008494   0.00461502 ..., -0.00058999  0.00131528
   0.00283094]
 [-0.00314315  0.0020217  -0.00460647 ..., -0.00474952  0.00424651
  -0.00164312]]
(3791, 100)


### docベクトルを確認する

適当な文書を選んでベクトルを見てみる

In [9]:
docvec = model.docvecs['../data/text/dokujo-tsushin/dokujo-tsushin-4887920.txt']
print(docvec)

[-0.13919863 -0.43473208  0.40603662 -0.16864267  0.17161784  0.19547234
 -0.65599167  0.18168986 -0.0247904   0.57588536 -0.32652783 -0.14116621
  0.12158963  0.31824687 -0.33371785  0.01724834 -0.31305146 -0.2392212
  0.49822816  0.18790688 -0.13369435 -0.27870515  0.17312247  0.41790166
  0.25199282 -0.0926208  -0.29965529 -0.12984681  0.02414287  0.16712297
  0.27444375 -0.14860223 -0.03739775  0.36650175 -0.73449528  0.2937839
  0.24706443  0.2010209  -0.30464229  0.12827997 -0.10724422  0.30067831
 -0.05517422  0.23755911  0.90899724 -0.52097768  0.15882584 -0.16939884
  0.04678448 -0.15268083  0.07954071 -0.18549587  0.08620507  0.16964303
  0.22955421 -0.00382834  0.37277809  0.14150985 -0.02264774  0.20699537
  0.68228108  0.03019632  0.52654952 -0.04512729  0.16508335  0.06000083
 -0.36324218  0.32735533  0.10777961 -0.35875541  0.23353495 -0.03063663
  0.13637739  0.37539837 -0.091947    0.05131961 -0.04122614 -0.06674042
  0.27040523 -0.16008213 -0.11133077 -0.0582763   0.4

### tensorflowに使える形式でベクトルを保存する


In [10]:
import tensorflow as tf

fp = FloatProgress(min=0, max=len(dir_docs))
display(fp)
step = 1
fp.value = step

with tf.python_io.TFRecordWriter('../input/train.tfr') as x_writer, tf.python_io.TFRecordWriter('../input/test.tfr') as t_writer:
    for doc in dir_docs:
        if (doc[0] not in model.docvecs):
            continue
        example = tf.train.Example(features=tf.train.Features(feature={
            'id': tf.train.Feature(int64_list=tf.train.Int64List(value=[doc[1]])),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[doc[3]])),
            'feature': tf.train.Feature(float_list=tf.train.FloatList(value=model.docvecs[doc[0]]))
        }))
        # print(example.SerializeToString())
        
        # 9割のデータを学習用に使う
        if (random.randint(1,100) < 90):
            x_writer.write(example.SerializeToString())
        else:
            t_writer.write(example.SerializeToString())
        
        step += 1
        fp.value = step

# model.save('../input/doc2vec.model')

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:15: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding
